In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib as plt
import statsmodels.api as sm
import statsmodels.stats.api as sms
import seaborn as sns
import scipy.stats as scipy
from scipy.stats import shapiro

In [2]:
df = pd.read_csv("/work/Motor_Vehicle_Collisions_-_Crashes.csv")

/tmp/ipykernel_283/2334064192.py:1: DtypeWarning: Columns (4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/work/Motor_Vehicle_Collisions_-_Crashes.csv")


In [3]:
df = df.dropna(subset=["VEHICLE_MAKE", 'DRIVER_LICENSE_STATUS','DRIVER_SEX', 'VEHICLE_YEAR'])
df = df[df['VEHICLE_YEAR'] <= 2025]
df = df[df['VEHICLE_OCCUPANTS'] <= 20]

In [4]:
#are Ford vehicles more prone to fatal accidents
df

,UNIQUE_ID,COLLISION_ID,CRASH_DATE,CRASH_TIME,VEHICLE_ID,STATE_REGISTRATION,VEHICLE_TYPE,VEHICLE_MAKE,VEHICLE_MODEL,VEHICLE_YEAR,...,VEHICLE_DAMAGE,VEHICLE_DAMAGE_1,VEHICLE_DAMAGE_2,VEHICLE_DAMAGE_3,PUBLIC_PROPERTY_DAMAGE,PUBLIC_PROPERTY_DAMAGE_TYPE,CONTRIBUTING_FACTOR_1,CONTRIBUTING_FACTOR_2,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED
2,16937807,3405925,3/22/2016,5:40,161407,NY,NaN,FORD -CAR/SUV,NaN,1997.0,...,Center Front End,NaN,NaN,NaN,N,NaN,Outside Car Distraction,Driver Inattention/Distraction,0,0
4,16946723,3406521,3/24/2016,15:30,38027,CT,NaN,BMW -CAR/SUV,NaN,2013.0,...,Left Front Bumper,NaN,NaN,NaN,N,NaN,Unsafe Speed,Following Too Closely,0,0
5,16946721,3406521,3/24/2016,15:30,38026,NY,NaN,HYUN -CAR/SUV,NaN,2013.0,...,Right Rear Bumper,Center Front End,NaN,NaN,Unspecified,NaN,Unspecified,Unspecified,0,0
6,16947454,3406526,3/25/2016,12:20,50680,NY,Taxi,FORD -CAR/SUV,FORD ZZZ,2011.0,...,Right Front Quarter Panel,NaN,NaN,NaN,N,NaN,Passing Too Closely,NaN,0,0
7,16972534,3406722,4/15/2016,21:40,156014,NY,NaN,CHEV -CAR/SUV,NaN,2014.0,...,Right Front Quarter Panel,Right Front Bumper,NaN,NaN,N,NaN,Unsafe Lane Changing,Unspecified,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89092,17311768,3518719,9/11/2016,20:30,778066,NY,Station Wagon/Sport Utility Vehicle,FORD -CAR/SUV,NaN,2016.0,...,Center Front End,Left Side Doors,No Damage,No Damage,N,NaN,Traffic Control Disregarded,Failure to Yield Right-of-Way,11,0
89093,17414543,3578653,12/6/2016,3:00,1189706,NJ,Sedan,JEEP -CAR/SUV,NaN,2013.0,...,Center Back End,Right Rear Bumper,Left Rear Bumper,Undercarriage,N,NaN,Alcohol Involvement,Unsafe Speed,11,0
89094,17414544,3578653,12/6/2016,3:00,1189705,NJ,Sedan,HOND -CAR/SUV,NaN,2008.0,...,Center Back End,Left Rear Quarter Panel,NaN,NaN,N,NaN,Alcohol Involvement,Unsafe Speed,11,0
89095,17414542,3578653,12/6/2016,3:00,1189702,NY,Sedan,MERZ -CAR/SUV,NaN,2016.0,...,Center Front End,Left Front Bumper,Right Front Quarter Panel,Right Front Bumper,N,NaN,Alcohol Involvement,Unsafe Speed,11,0


In [5]:
df = df[['CRASH_DATE', 'CRASH_TIME', 'VEHICLE_MAKE', 'VEHICLE_YEAR','VEHICLE_OCCUPANTS', 'DRIVER_SEX', 'DRIVER_LICENSE_STATUS', 
        'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED']].copy()

In [6]:
df = df.drop(df[df['NUMBER OF PERSONS INJURED'] > df['VEHICLE_OCCUPANTS']].index)

In [7]:
df

,CRASH_DATE,CRASH_TIME,VEHICLE_MAKE,VEHICLE_YEAR,VEHICLE_OCCUPANTS,DRIVER_SEX,DRIVER_LICENSE_STATUS,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED
2,3/22/2016,5:40,FORD -CAR/SUV,1997.0,1.0,M,Licensed,0,0
4,3/24/2016,15:30,BMW -CAR/SUV,2013.0,1.0,M,Licensed,0,0
5,3/24/2016,15:30,HYUN -CAR/SUV,2013.0,1.0,M,Licensed,0,0
6,3/25/2016,12:20,FORD -CAR/SUV,2011.0,1.0,M,Licensed,0,0
7,4/15/2016,21:40,CHEV -CAR/SUV,2014.0,1.0,M,Licensed,0,0
...,...,...,...,...,...,...,...,...,...
89071,12/1/2016,21:04,CHEVY,2015.0,7.0,F,Licensed,7,0
89072,5/11/2018,21:15,GMC -CAR/SUV,2013.0,7.0,M,Licensed,7,0
89084,10/6/2016,8:55,FORD -CAR/SUV,2012.0,10.0,M,Licensed,9,0
89085,10/11/2016,0:00,FORD -CAR/SUV,2014.0,9.0,F,Licensed,9,0


In [8]:
df['isFord'] = df['VEHICLE_MAKE'].apply(lambda x: 1 if isinstance(x, str) and 'FORD' in x.upper() else 0)
df['Fatal'] = df['NUMBER OF PERSONS KILLED'].apply(lambda x: 1 if x > 0 else 0)
df['LicenseStatus'] = df['DRIVER_LICENSE_STATUS'].apply(lambda x: 1 if 'Licensed' in x else 0)

In [9]:
df['Fatal'].value_counts()

Fatal
0    62517
1       36
Name: count, dtype: int64

In [10]:
df['NUMBER OF PERSONS KILLED'].value_counts()

NUMBER OF PERSONS KILLED
0    62517
1       35
3        1
Name: count, dtype: int64

In [11]:
df['DRIVER_LICENSE_STATUS'].value_counts()

DRIVER_LICENSE_STATUS
Licensed      61411
Unlicensed      820
Permit          322
Name: count, dtype: int64

In [12]:
df['VEHICLE_MAKE'].value_counts().head()

VEHICLE_MAKE
TOYT -CAR/SUV    11652
HOND -CAR/SUV     7134
NISS -CAR/SUV     6382
FORD -CAR/SUV     5725
CHEV -CAR/SUV     3146
Name: count, dtype: int64

In [13]:
df.value_counts(df['VEHICLE_MAKE'].str.contains('FORD') & df['Fatal'] == 1)

False    62545
True         8
Name: count, dtype: int64

In [14]:
df.value_counts(df['NUMBER OF PERSONS INJURED'] > 0)

NUMBER OF PERSONS INJURED
False    52547
True     10006
Name: count, dtype: int64

In [15]:
df.value_counts(df['NUMBER OF PERSONS INJURED'])

NUMBER OF PERSONS INJURED
0     52547
1      8617
2      1055
3       226
4        68
5        27
6         8
7         2
9         2
10        1
Name: count, dtype: int64

In [16]:
df.value_counts(df['VEHICLE_OCCUPANTS'] > 0)

VEHICLE_OCCUPANTS
True     59855
False     2698
Name: count, dtype: int64

In [53]:
df['VEHICLE_MAKE'].value_counts().to_csv('vehicle_counts.csv')

In [18]:
df[df['Fatal'] == 1]['VEHICLE_MAKE'].value_counts()

VEHICLE_MAKE
FORD -CAR/SUV      5
TOYT -CAR/SUV      5
CHEV -CAR/SUV      3
FORD               2
MERZ -CAR/SUV      2
GMC -CAR/SUV       2
HOND -CAR/SUV      2
SUZI -MCL          1
BMW -CAR/SUV       1
YAMA -MCL          1
MAZD -CAR/SUV      1
FORD-TRUCK/BUS     1
INTL-TRUCK/BUS     1
HYUN -CAR/SUV      1
ACUR -CAR/SUV      1
AUDI -CAR/SUV      1
VOLV -TRUCK/BUS    1
MERC -CAR/SUV      1
MACK-TRUCK/BUS     1
NE/FL              1
HOND -MCL          1
CHRY -CAR/SUV      1
Name: count, dtype: int64

In [19]:
df['Injured'] = df['NUMBER OF PERSONS INJURED'].apply(lambda x: 1 if x > 0 else 0)

In [20]:
df

,CRASH_DATE,CRASH_TIME,VEHICLE_MAKE,VEHICLE_YEAR,VEHICLE_OCCUPANTS,DRIVER_SEX,DRIVER_LICENSE_STATUS,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,isFord,Fatal,LicenseStatus,Injured
2,3/22/2016,5:40,FORD -CAR/SUV,1997.0,1.0,M,Licensed,0,0,1,0,1,0
4,3/24/2016,15:30,BMW -CAR/SUV,2013.0,1.0,M,Licensed,0,0,0,0,1,0
5,3/24/2016,15:30,HYUN -CAR/SUV,2013.0,1.0,M,Licensed,0,0,0,0,1,0
6,3/25/2016,12:20,FORD -CAR/SUV,2011.0,1.0,M,Licensed,0,0,1,0,1,0
7,4/15/2016,21:40,CHEV -CAR/SUV,2014.0,1.0,M,Licensed,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89071,12/1/2016,21:04,CHEVY,2015.0,7.0,F,Licensed,7,0,0,0,1,1
89072,5/11/2018,21:15,GMC -CAR/SUV,2013.0,7.0,M,Licensed,7,0,0,0,1,1
89084,10/6/2016,8:55,FORD -CAR/SUV,2012.0,10.0,M,Licensed,9,0,1,0,1,1
89085,10/11/2016,0:00,FORD -CAR/SUV,2014.0,9.0,F,Licensed,9,0,1,0,1,1


In [21]:
logit_y = df['Fatal']

In [22]:
#logit model
x = df['isFord']
x = sm.add_constant(x)
model1 = sm.Logit(logit_y,x).fit()

Optimization terminated successfully.
         Current function value: 0.004835
         Iterations 12


In [23]:
results_model1 = model1.summary()
results_model1

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  Fatal   No. Observations:                62553
Model:                          Logit   Df Residuals:                    62551
Method:                           MLE   Df Model:                            1
Date:                Tue, 02 Dec 2025   Pseudo R-squ.:                0.006954
Time:                        17:00:40   Log-Likelihood:                -302.44
converged:                       True   LL-Null:                       -304.56
Covariance Type:            nonrobust   LLR p-value:                   0.03958
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -7.6013      0.189    -40.213      0.000      -7.972      -7.231
isFord         0.9025      0.401      2.250      0.024       0.116       1.689
==============================================================================
"""

In [24]:
df['date'] = pd.to_datetime(df['CRASH_DATE'])
df['time'] = pd.to_datetime(df['CRASH_TIME'])
df['month'] = df['date'].dt.strftime('%m').astype(int)
df['day'] = df['date'].dt.strftime("%A")
df['time'] = df['time'].dt.strftime('%H%M').astype(int)
df

/tmp/ipykernel_283/3078422805.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['time'] = pd.to_datetime(df['CRASH_TIME'])


,CRASH_DATE,CRASH_TIME,VEHICLE_MAKE,VEHICLE_YEAR,VEHICLE_OCCUPANTS,DRIVER_SEX,DRIVER_LICENSE_STATUS,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,isFord,...,date,time,month,day,peakTime,weekday,springSummer,isHonda,isToyota,driverSex
2,3/22/2016,5:40,FORD -CAR/SUV,1997.0,1.0,M,Licensed,0,0,1,...,2016-03-22,540,3,Tuesday,0,1,1,0,0,1
4,3/24/2016,15:30,BMW -CAR/SUV,2013.0,1.0,M,Licensed,0,0,0,...,2016-03-24,1530,3,Thursday,0,1,1,0,0,1
5,3/24/2016,15:30,HYUN -CAR/SUV,2013.0,1.0,M,Licensed,0,0,0,...,2016-03-24,1530,3,Thursday,0,1,1,0,0,1
6,3/25/2016,12:20,FORD -CAR/SUV,2011.0,1.0,M,Licensed,0,0,1,...,2016-03-25,1220,3,Friday,0,1,1,0,0,1
7,4/15/2016,21:40,CHEV -CAR/SUV,2014.0,1.0,M,Licensed,0,0,0,...,2016-04-15,2140,4,Friday,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89071,12/1/2016,21:04,CHEVY,2015.0,7.0,F,Licensed,7,0,0,...,2016-12-01,2104,12,Thursday,0,1,0,0,0,0
89072,5/11/2018,21:15,GMC -CAR/SUV,2013.0,7.0,M,Licensed,7,0,0,...,2018-05-11,2115,5,Friday,0,1,1,0,0,1
89084,10/6/2016,8:55,FORD -CAR/SUV,2012.0,10.0,M,Licensed,9,0,1,...,2016-10-06,855,10,Thursday,1,1,0,0,0,1
89085,10/11/2016,0:00,FORD -CAR/SUV,2014.0,9.0,F,Licensed,9,0,1,...,2016-10-11,0,10,Tuesday,0,1,0,0,0,0


In [59]:
df['peakTime'] = df['time'].apply(lambda x: 1 if x >= 700 and x <= 900 else(1 if x >= 1600 and x <= 1900 else 0))
df['weekday'] = df['day'].apply(lambda x: 1 if x != 'Saturday' or x != 'Sunday' else 0)
df['springSummer'] = df['month'].apply(lambda x: 1 if x <= 8 else 0)
df['isCar'] = df['VEHICLE_MAKE'].apply(lambda x: 1 if isinstance(x, str) and 'CAR'in x.upper() else 0)

In [62]:
df

,CRASH_DATE,CRASH_TIME,VEHICLE_MAKE,VEHICLE_YEAR,VEHICLE_OCCUPANTS,DRIVER_SEX,DRIVER_LICENSE_STATUS,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,isFord,...,time,month,day,peakTime,weekday,springSummer,isHonda,isToyota,driverSex,isCar
2,3/22/2016,5:40,FORD -CAR/SUV,1997.0,1.0,M,Licensed,0,0,1,...,540,3,Tuesday,0,1,1,0,0,1,1
4,3/24/2016,15:30,BMW -CAR/SUV,2013.0,1.0,M,Licensed,0,0,0,...,1530,3,Thursday,0,1,1,0,0,1,1
5,3/24/2016,15:30,HYUN -CAR/SUV,2013.0,1.0,M,Licensed,0,0,0,...,1530,3,Thursday,0,1,1,0,0,1,1
6,3/25/2016,12:20,FORD -CAR/SUV,2011.0,1.0,M,Licensed,0,0,1,...,1220,3,Friday,0,1,1,0,0,1,1
7,4/15/2016,21:40,CHEV -CAR/SUV,2014.0,1.0,M,Licensed,0,0,0,...,2140,4,Friday,0,1,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89071,12/1/2016,21:04,CHEVY,2015.0,7.0,F,Licensed,7,0,0,...,2104,12,Thursday,0,1,0,0,0,0,0
89072,5/11/2018,21:15,GMC -CAR/SUV,2013.0,7.0,M,Licensed,7,0,0,...,2115,5,Friday,0,1,1,0,0,1,1
89084,10/6/2016,8:55,FORD -CAR/SUV,2012.0,10.0,M,Licensed,9,0,1,...,855,10,Thursday,1,1,0,0,0,1,1
89085,10/11/2016,0:00,FORD -CAR/SUV,2014.0,9.0,F,Licensed,9,0,1,...,0,10,Tuesday,0,1,0,0,0,0,1


In [27]:
df['isHonda'] = df['VEHICLE_MAKE'].apply(lambda x : 1 if isinstance(x, str) and 'HON' in x.upper() else 0)
df['isToyota'] = df['VEHICLE_MAKE'].apply(lambda x : 1 if isinstance(x, str) and 'TOY' in x.upper() else 0)

In [28]:
df['driverSex'] = df['DRIVER_SEX'].apply(lambda x: 1 if x == 'M' else 0)
df

,CRASH_DATE,CRASH_TIME,VEHICLE_MAKE,VEHICLE_YEAR,VEHICLE_OCCUPANTS,DRIVER_SEX,DRIVER_LICENSE_STATUS,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,isFord,...,date,time,month,day,peakTime,weekday,springSummer,isHonda,isToyota,driverSex
2,3/22/2016,5:40,FORD -CAR/SUV,1997.0,1.0,M,Licensed,0,0,1,...,2016-03-22,540,3,Tuesday,0,1,1,0,0,1
4,3/24/2016,15:30,BMW -CAR/SUV,2013.0,1.0,M,Licensed,0,0,0,...,2016-03-24,1530,3,Thursday,0,1,1,0,0,1
5,3/24/2016,15:30,HYUN -CAR/SUV,2013.0,1.0,M,Licensed,0,0,0,...,2016-03-24,1530,3,Thursday,0,1,1,0,0,1
6,3/25/2016,12:20,FORD -CAR/SUV,2011.0,1.0,M,Licensed,0,0,1,...,2016-03-25,1220,3,Friday,0,1,1,0,0,1
7,4/15/2016,21:40,CHEV -CAR/SUV,2014.0,1.0,M,Licensed,0,0,0,...,2016-04-15,2140,4,Friday,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89071,12/1/2016,21:04,CHEVY,2015.0,7.0,F,Licensed,7,0,0,...,2016-12-01,2104,12,Thursday,0,1,0,0,0,0
89072,5/11/2018,21:15,GMC -CAR/SUV,2013.0,7.0,M,Licensed,7,0,0,...,2018-05-11,2115,5,Friday,0,1,1,0,0,1
89084,10/6/2016,8:55,FORD -CAR/SUV,2012.0,10.0,M,Licensed,9,0,1,...,2016-10-06,855,10,Thursday,1,1,0,0,0,1
89085,10/11/2016,0:00,FORD -CAR/SUV,2014.0,9.0,F,Licensed,9,0,1,...,2016-10-11,0,10,Tuesday,0,1,0,0,0,0


In [65]:
x2 = df[['isFord', 'peakTime', 'weekday', 'VEHICLE_YEAR', 'LicenseStatus', 'driverSex', 'VEHICLE_OCCUPANTS', 
        'springSummer', 'isCar']]
x2 = sm.add_constant(x2)

In [66]:
#complex logit model
model3 = sm.Logit(logit_y, x2).fit()
results_model3 = model3.summary()
results_model3

Optimization terminated successfully.
         Current function value: 0.004613
         Iterations 14


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  Fatal   No. Observations:                62553
Model:                          Logit   Df Residuals:                    62544
Method:                           MLE   Df Model:                            8
Date:                Tue, 02 Dec 2025   Pseudo R-squ.:                 0.05261
Time:                        17:13:04   Log-Likelihood:                -288.54
converged:                       True   LL-Null:                       -304.56
Covariance Type:            nonrobust   LLR p-value:                 9.140e-05
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
isFord                0.8437      0.403      2.092      0.036       0.053       1.634
peakTime             -1.3632      0.531     -2.568      0.010      -2.404      -0.323
weekday               2.1756     58.644      0.037      0.970    -112.764     117.115
VEHICLE_YEAR         -0.0032      0.029     -0.111      0.912      -0.060       0.054
LicenseStatus        -1.9072      0.538     -3.546      0.000      -2.961      -0.853
driverSex            -0.2291      0.398     -0.576      0.565      -1.009       0.551
VEHICLE_OCCUPANTS     0.0137      0.144      0.095      0.924      -0.269       0.296
springSummer          0.5563      0.483      1.152      0.249      -0.390       1.502
isCar                -1.3220      0.384     -3.444      0.001      -2.074      -0.570
=====================================================================================
"""

In [77]:
x3 = df[['isFord', 'isToyota', 'isHonda', 'peakTime', 'weekday', 'VEHICLE_YEAR', 'LicenseStatus', 
            'driverSex', 'VEHICLE_OCCUPANTS', 'springSummer', 'isCar']]
x3 = sm.add_constant(x3)


In [78]:
#logit model with other makes
model5 = sm.Logit(logit_y, x3).fit()
model5_results = model5.summary()
model5_results

Optimization terminated successfully.
         Current function value: 0.004612
         Iterations 14


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  Fatal   No. Observations:                62553
Model:                          Logit   Df Residuals:                    62542
Method:                           MLE   Df Model:                           10
Date:                Tue, 02 Dec 2025   Pseudo R-squ.:                 0.05268
Time:                        17:14:00   Log-Likelihood:                -288.51
converged:                       True   LL-Null:                       -304.56
Covariance Type:            nonrobust   LLR p-value:                 0.0003872
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
isFord                0.8632      0.420      2.053      0.040       0.039       1.687
isToyota              0.1074      0.522      0.206      0.837      -0.915       1.130
isHonda               0.0199      0.633      0.031      0.975      -1.220       1.260
peakTime             -1.3623      0.531     -2.566      0.010      -2.403      -0.322
weekday               3.0381     58.780      0.052      0.959    -112.169     118.245
VEHICLE_YEAR         -0.0037      0.029     -0.125      0.900      -0.061       0.054
LicenseStatus        -1.9110      0.538     -3.550      0.000      -2.966      -0.856
driverSex            -0.2340      0.399     -0.586      0.558      -1.016       0.548
VEHICLE_OCCUPANTS     0.0139      0.144      0.096      0.923      -0.268       0.296
springSummer          0.5568      0.483      1.153      0.249      -0.389       1.503
isCar                -1.3432      0.403     -3.336      0.001      -2.132      -0.554
=====================================================================================
"""

In [79]:
#negative binomial model
y = df['NUMBER OF PERSONS KILLED']
nb_model = sm.GLM(y, x3, family = sm.families.NegativeBinomial()).fit()
nb_results = nb_model.summary()
nb_results

/root/venv/lib/python3.11/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


<class 'statsmodels.iolib.summary.Summary'>
"""
                    Generalized Linear Model Regression Results                     
====================================================================================
Dep. Variable:     NUMBER OF PERSONS KILLED   No. Observations:                62553
Model:                                  GLM   Df Residuals:                    62542
Model Family:              NegativeBinomial   Df Model:                           10
Link Function:                          Log   Scale:                          1.0000
Method:                                IRLS   Log-Likelihood:                -301.56
Date:                      Tue, 02 Dec 2025   Deviance:                       501.57
Time:                              17:14:01   Pearson chi2:                 6.84e+04
No. Iterations:                          11   Pseudo R-squ. (CS):          0.0005719
Covariance Type:                  nonrobust                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
isFord                1.0619      0.390      2.726      0.006       0.298       1.825
isToyota              0.0787      0.519      0.152      0.879      -0.939       1.096
isHonda              -0.0091      0.631     -0.014      0.989      -1.246       1.228
peakTime             -1.4313      0.529     -2.705      0.007      -2.469      -0.394
weekday              21.2870     55.828      0.381      0.703     -88.134     130.708
VEHICLE_YEAR         -0.0129      0.028     -0.466      0.642      -0.067       0.042
LicenseStatus        -1.7967      0.538     -3.337      0.001      -2.852      -0.742
driverSex            -0.1295      0.394     -0.329      0.742      -0.901       0.642
VEHICLE_OCCUPANTS     0.1142      0.099      1.154      0.249      -0.080       0.308
springSummer          0.4770      0.481      0.991      0.322      -0.467       1.421
isCar                -1.1959      0.397     -3.015      0.003      -1.973      -0.418
=====================================================================================
"""

In [80]:
logit_injured_y = df['Injured']
linear_injured_y = df['NUMBER OF PERSONS INJURED']

In [81]:
#injured logit model
injured_model = sm.Logit(logit_injured_y, x).fit()
injured_model_summary = injured_model.summary()
injured_model_summary

Optimization terminated successfully.
         Current function value: 0.439602
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                Injured   No. Observations:                62553
Model:                          Logit   Df Residuals:                    62551
Method:                           MLE   Df Model:                            1
Date:                Tue, 02 Dec 2025   Pseudo R-squ.:               3.783e-06
Time:                        17:14:03   Log-Likelihood:                -27498.
converged:                       True   LL-Null:                       -27499.
Covariance Type:            nonrobust   LLR p-value:                    0.6483
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.6568      0.012   -143.874      0.000      -1.679      -1.634
isFord        -0.0164      0.036     -0.455      0.649      -0.087       0.054
==============================================================================
"""

No statistical significance that ford cars cause more fatal accidents. fatal accidents are rare cannot be truly determined with OLS and Logit model

In [82]:
#injured logit model with other makes
injured_model2 = sm.Logit(logit_injured_y, x3).fit()
injured_model_summary2 = injured_model2.summary()
injured_model_summary2

Optimization terminated successfully.
         Current function value: 0.432137
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                Injured   No. Observations:                62553
Model:                          Logit   Df Residuals:                    62542
Method:                           MLE   Df Model:                           10
Date:                Tue, 02 Dec 2025   Pseudo R-squ.:                 0.01699
Time:                        17:14:04   Log-Likelihood:                -27031.
converged:                       True   LL-Null:                       -27499.
Covariance Type:            nonrobust   LLR p-value:                2.792e-194
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
isFord               -0.0240      0.037     -0.641      0.522      -0.097       0.049
isToyota              0.0756      0.029      2.579      0.010       0.018       0.133
isHonda               0.0092      0.035      0.260      0.795      -0.060       0.078
peakTime             -0.0550      0.024     -2.331      0.020      -0.101      -0.009
weekday              30.5547      3.911      7.812      0.000      22.889      38.221
VEHICLE_YEAR         -0.0161      0.002     -8.286      0.000      -0.020      -0.012
LicenseStatus        -0.4749      0.071     -6.731      0.000      -0.613      -0.337
driverSex            -0.0320      0.025     -1.285      0.199      -0.081       0.017
VEHICLE_OCCUPANTS     0.2612      0.010     26.997      0.000       0.242       0.280
springSummer          0.1027      0.038      2.697      0.007       0.028       0.177
isCar                 0.3830      0.043      8.927      0.000       0.299       0.467
=====================================================================================
"""

In [83]:
#linear injured model with other makes
linear_injured_model2 = sm.OLS(linear_injured_y, x3).fit()
linear_injured_summary2 = linear_injured_model2.summary()
linear_injured_summary2

<class 'statsmodels.iolib.summary.Summary'>
"""
                                OLS Regression Results                               
=====================================================================================
Dep. Variable:     NUMBER OF PERSONS INJURED   R-squared:                       0.048
Model:                                   OLS   Adj. R-squared:                  0.048
Method:                        Least Squares   F-statistic:                     314.7
Date:                       Tue, 02 Dec 2025   Prob (F-statistic):               0.00
Time:                               17:14:04   Log-Likelihood:                -42598.
No. Observations:                      62553   AIC:                         8.522e+04
Df Residuals:                          62542   BIC:                         8.532e+04
Df Model:                                 10                                         
Covariance Type:                   nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
isFord                0.0024      0.006      0.378      0.705      -0.010       0.015
isToyota              0.0141      0.005      2.717      0.007       0.004       0.024
isHonda               0.0035      0.006      0.564      0.573      -0.009       0.016
peakTime             -0.0142      0.004     -3.489      0.000      -0.022      -0.006
weekday               5.0235      0.697      7.203      0.000       3.657       6.390
VEHICLE_YEAR         -0.0025      0.000     -7.099      0.000      -0.003      -0.002
LicenseStatus        -0.0739      0.014     -5.161      0.000      -0.102      -0.046
driverSex            -0.0058      0.004     -1.316      0.188      -0.014       0.003
VEHICLE_OCCUPANTS     0.1052      0.002     54.154      0.000       0.101       0.109
springSummer          0.0143      0.007      2.091      0.037       0.001       0.028
isCar                 0.0630      0.007      9.434      0.000       0.050       0.076
==============================================================================
Omnibus:                    43343.688   Durbin-Watson:                   0.091
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           883643.790
Skew:                           3.099   Prob(JB):                         0.00
Kurtosis:                      20.338   Cond. No.                     7.33e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.33e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [84]:
#negative binomial model for injured
nb2_model = sm.GLM(linear_injured_y, x3, family = sm.families.NegativeBinomial()).fit()
nb2_results = nb2_model.summary()
nb2_results

/root/venv/lib/python3.11/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Generalized Linear Model Regression Results                     
=====================================================================================
Dep. Variable:     NUMBER OF PERSONS INJURED   No. Observations:                62553
Model:                                   GLM   Df Residuals:                    62542
Model Family:               NegativeBinomial   Df Model:                           10
Link Function:                           Log   Scale:                          1.0000
Method:                                 IRLS   Log-Likelihood:                -31581.
Date:                       Tue, 02 Dec 2025   Deviance:                       33660.
Time:                               17:14:05   Pearson chi2:                 5.22e+04
No. Iterations:                           15   Pseudo R-squ. (CS):            0.03570
Covariance Type:                   nonrobust                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
isFord               -0.0017      0.034     -0.050      0.960      -0.069       0.065
isToyota              0.0683      0.027      2.510      0.012       0.015       0.122
isHonda               0.0176      0.032      0.541      0.589      -0.046       0.081
peakTime             -0.0727      0.022     -3.315      0.001      -0.116      -0.030
weekday              24.7129      3.615      6.836      0.000      17.627      31.799
VEHICLE_YEAR         -0.0134      0.002     -7.452      0.000      -0.017      -0.010
LicenseStatus        -0.3491      0.064     -5.413      0.000      -0.475      -0.223
driverSex            -0.0363      0.023     -1.591      0.112      -0.081       0.008
VEHICLE_OCCUPANTS     0.3724      0.007     56.459      0.000       0.359       0.385
springSummer          0.0722      0.035      2.054      0.040       0.003       0.141
isCar                 0.4248      0.041     10.467      0.000       0.345       0.504
=====================================================================================
"""

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f2ce982c-ab7e-4df7-957e-2768a79f0182' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>